# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key as gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../output_data/cities.csv')
cities.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)

In [4]:
coordinates = cities.loc[:,['Lat', 'Lng']]
coordinates = coordinates.fillna(0)
humidity = cities.loc[:,['Humidity']]
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity['Humidity'], dissipating=False, max_intensity= 10, point_radius=0.5)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Eliminate non ideal weather conditions
prio_df = pd.DataFrame(cities, columns = ['City', 'Cloudiness', 'Max Temp', 'Wind Speed', 'Lat', 'Lng', 'Country'])
prio_df = prio_df.dropna()
ideal_wind = (prio_df['Wind Speed'] < 6)
ideal_cloud = (prio_df['Cloudiness'] == 0)
ideal_temp = (prio_df['Max Temp'] <=80) & (prio_df['Max Temp'] >= 70)

ideal_df = prio_df[ideal_cloud & ideal_temp & ideal_wind]

ideal_df

,City,Cloudiness,Max Temp,Wind Speed,Lat,Lng,Country
70,ponta do sol,0,73.68,5.32,-20.63,-46.00,BR
88,cidreira,0,76.56,5.50,-30.17,-50.22,BR
139,vaini,0,78.90,1.59,15.34,74.49,IN
297,mogok,0,72.24,2.77,22.92,96.51,MM


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(cities, columns = ['City', 'Country', 'Lat', 'Lng'])
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [17]:
hotel = []
params = {"radius": 5000, "types": "hotel", "keyword": "hotel", "key": gkey}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params['location']= f'{lat},{lng}'
    map_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    address = requests.get(map_url, params=params)
    address = address.json()
    try:
        hotel.append(address['results'][0]['name'])
    except IndexError:
        hotel.append(np.nan)

In [ ]:
hotel 

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig